# Joining Data with SQL

The following is a summary of the work accomplished as part of a [course on Datacamp](https://campus.datacamp.com/courses/joining-data-in-sql/)

## Join: `INNER`, `LEFT`, `RIGHT`, `FULL`, `CROSS`

`Inner`: both left and right tables must have non-null values

`Left`: the first table must have complete data for that row to be carried forward

`Right`: the second table must have complete data for it to be carried

`Full`: both tables must have

`Cross`: no need to stipulate a column on which to join, because all possible combinations are outputted


### **Objective:** We are interested in pulling information about the countries, languages and currencies.
Data which exists across 3 tables.


In [2]:
-- Since `languages` and `currencies` exist in separate tables, this will require two consecutive full joins involving the `countries`, `languages` and `currencies` tables.
SELECT
	c1.name AS country,
    region,
    l.name AS language,
	basic_unit,
    frac_unit
FROM world.countries as c1
-- Full join with languages (alias as l)
	FULL JOIN world.languages as l
		USING (code)
	-- Full join with currencies (alias as c2)
	FULL JOIN world.currencies as c2
		USING (code)
WHERE region LIKE 'M%esia'
LIMIT 8;

,country,region,language,basic_unit,frac_unit
0,Guam,Micronesia,English,None,None
1,Guam,Micronesia,Filipino,None,None
2,Guam,Micronesia,Chamorro,None,None
3,Guam,Micronesia,Other Pacific Islander,None,None
4,Guam,Micronesia,Asian,None,None
5,Guam,Micronesia,Other,None,None
6,Kiribati,Micronesia,Kiribati,Australian dollar,Cent
7,Kiribati,Micronesia,English,Australian dollar,Cent


### **Objective:** compare with itself. 

By joining the table with itself, we can read multiple rows. We can then transform these entries into columns using Alias.

In [3]:
-- Alias size 2 times
SELECT
	p1.country_code,
    p1.size AS size2010,
    p2.size AS size2015
FROM world.populations AS p1
INNER JOIN world.populations AS p2
ON p1.country_code = p2.country_code
WHERE
	p1.year = 2010
-- Filter such that p1.year is always five years before p2.year
    AND p2.year = p1.year + 5
LIMIT 10;

,country_code,size2010,size2015
0,ABW,101597,103889
1,AFG,27962208,32526562
2,AGO,21219954,25021974
3,ALB,2913021,2889167
4,AND,84419,70473
5,ARE,8329453,9156963
6,ARG,41222876,43416756
7,ARM,2963496,3017712
8,ASM,55636,55538
9,ATG,87233,91818


## WHERE Subqueries:

### **Objective:** We are interested in analyzing `inflation` and `unemployment` rate for certain countries in 2015. 

We are not interested in countries with a form of governemnt that includes the terms `Republic` or `Monarchy`, but are interested in all other forms of government.

In [4]:
-- Select relevant fields
SELECT code, inflation_rate, unemployment_rate
FROM world.economies
WHERE year = 2015
  AND code NOT IN -- Subquery returning country codes filtered on gov_form
	(SELECT code
  	FROM world.countries
  	WHERE gov_form LIKE '%Monarchy%' OR gov_form LIKE '%Republic%')
ORDER BY inflation_rate
LIMIT 10;

,code,inflation_rate,unemployment_rate
0,AFG,-1.549,NaN
1,CHE,-1.140,3.178
2,PRI,-0.751,12.000
3,ROU,-0.596,6.812
4,TLS,0.553,NaN
5,MNE,1.204,NaN
6,SRB,1.392,18.200
7,HKG,3.037,3.296
8,ARE,4.070,NaN
9,MAC,4.564,1.825


### **Objective:** Determine the top 10 capital cities in `Europe` and the `Americas` by `city_perc`, a calculated metric. 
`city_perc` is a percentage that calculates the `proper` population in a city as a percentage of the total population in the wider `metro` area.

In [4]:
-- Select fields from cities
SELECT name, country_code, city_proper_pop, metroarea_pop,
    city_proper_pop / metroarea_pop * 100 AS city_perc
FROM world.cities

-- Use subquery to filter city name
WHERE name in
	(SELECT capital
    FROM world.countries
    WHERE continent = 'Europe' OR region LIKE '%America')

-- Add filter condition such that metroarea_pop does not have null values
AND metroarea_pop is not NULL

-- Sort and limit the result
ORDER BY city_perc DESC
LIMIT 10;

,name,country_code,city_proper_pop,metroarea_pop,city_perc
0,Lima,PER,8852000,10750000,82.344186
1,Bogota,COL,7878783,9800000,80.395746
2,Moscow,RUS,12197596,16170000,75.433493
3,Vienna,AUT,1863881,2600000,71.687728
4,Montevideo,URY,1305082,1947604,67.009616
5,Caracas,VEN,1943901,2923959,66.481817
6,Rome,ITA,2877215,4353775,66.085523
7,Brasilia,BRA,2556149,3919864,65.210146
8,London,GBR,8673713,13879757,62.491822
9,Budapest,HUN,1759407,2927944,60.090184


### The end